In [190]:
import json
import os,glob
import geonamescache
import numpy as np
import pandas as pd
import datetime
folder_path = 'D:\\COVID-19-TweetIDs\\2020-01\\New folder\\22'
gc = geonamescache.GeonamesCache()
df = pd.DataFrame()
df = pd.DataFrame(columns=("cityID",'time', 'cityname', 'state', 'countryname'))

In [191]:
def findloc(str):
    cities = str.split(",")[0].title() #use.title to make cities like hong kong to Hong Kong
    ct = gc.get_cities_by_name(cities)
    if (len(ct) == 1):
        return(ct[0]) #if return ct, it is a list of one member so lead to inconsistency of type
    elif ',' in str:
        loc2 = str.split(',')[1].strip()
        for c in ct:
            if(loc2 in json.dumps(c)) :
                return(c)

In [192]:
city = gc.get_cities()
city_items = city.items()
city_list = list(city_items)[0:10]
longlat = []
for k in city.keys():
    lst = []
    lst.append(city.get(k).get("longitude"))
    lst.append(city.get(k).get("latitude"))
    lst.append(city.get(k).get('geonameid'))
    longlat.append(lst)
tuple(longlat)
import math
def get_distance(data1, data2):
    points = zip(data1, data2)
    diffs_squared_distance = [pow(a - b, 2) for (a, b) in points]
    return math.sqrt(sum(diffs_squared_distance))
def findlonglat(a,b):
    found = []
    smallest = get_distance(b,a[0][0:2])
    for i in a:
        comparelist = i[0:2]
        if get_distance(b,comparelist) <= smallest:
            smallest = get_distance(b,comparelist)
            found = list(i)
    citycode = str(found[2])
    return {citycode:city[citycode]}

In [193]:
import csv

def countryName(countryInitial):
    country = csv.reader(open('countries.csv', 'r',encoding='utf-8'))
    for i in country:
        if(countryInitial == i[1]):
            return i[0]
        
def stateName(stateInitial):
    states = csv.reader(open('States.csv', 'r'))
    for i in states:
        if(stateInitial == i[2]):
            return i[0]

In [194]:
a = []
location = {}   # key is cityID, value is city information coming service
count = {}  # key is cityID and value is tweet count
date = {}

for filename in glob.glob(os.path.join(folder_path, '*.jsonl')):
    fhand = open(filename, 'r',encoding="UTF-8")
    i = 0
    for line in fhand:
        tweet = json.loads(line)
        try:
            user_name = tweet['user']["screen_name"]
            user_time = str(tweet["created_at"][:10]) + " " + str(tweet["created_at"][-4:])
            
            geo = tweet['coordinates']
            if geo != None: #Use coordinates to find location first
                geo = tweet['coordinates']["coordinates"]
                loc = findlonglat(longlat,geo)
            else: #Then try to use user's location
                loc = findloc(tweet['user']['location'])
            location.update(loc) #update into the location dictionary
            for key,value in loc.items():
                if(key not in count):
                    count.update({key:1})   #update in the count dictionary, for places with 1 tweet
                elif(key in count):
                    count.update({key:(count[key]+1)})     #update in the count dictionary, if cityID is repeated,just increase the value by 1       
            
            
            user_time = datetime.datetime.strptime(user_time, '%a %b %d %Y').strftime('%d/%m/%Y')#change the date
            user_city_name = loc[key]["name"]
            user_country_code = loc[key]['countrycode']
            user_country_name = countryName(user_country_code)#Change from initials into full country name
            user_state_code = loc[key]["admin1code"] if loc[key]["admin1code"].isalpha() else None
            user_state_name = stateName(user_state_code)
            [[key1,value1]] = loc.items()
            values_to_add = {"cityID": key1,'time': user_time,'cityname': user_city_name, 'state': user_state_name, 'countryname': user_country_name}

            
            row_to_add = pd.Series(values_to_add,name = "tweet")
            df = df.append(row_to_add)
        except:
            continue

df.to_csv('sample1.csv',encoding="UTF-8")


In [195]:
df.head()

,cityID,time,cityname,state,countryname
tweet,4140963,21/01/2020,Washington,District of Columbia,United States
tweet,4180439,21/01/2020,Atlanta,Georgia,United States
tweet,5380184,21/01/2020,Oxnard,California,United States
tweet,4364946,21/01/2020,Parkville,Maryland,United States
tweet,4887398,21/01/2020,Chicago,Illinois,United States


In [196]:
df.groupby(["cityID", "time","cityname","countryname"]).size().reset_index(name="count")

,cityID,time,cityname,countryname,count
0,100425,01/03/2020,Yanbu,Saudi Arabia,1
1,1005125,01/03/2020,Ermelo,South Africa,1
2,1006984,01/03/2020,East London,South Africa,5
3,1007311,01/03/2020,Durban,South Africa,18
4,1015621,01/03/2020,Brits,South Africa,1
...,...,...,...,...,...
2466,965301,01/03/2020,Pietermaritzburg,South Africa,2
2467,965528,01/03/2020,Phalaborwa,South Africa,1
2468,98182,01/03/2020,Baghdad,Iraq,1
2469,990930,01/03/2020,Kimberley,South Africa,2
